In [1]:
from skimage.transform import resize
import random
import numpy as np
import matplotlib.pyplot as plt
import math
import random

# Catch Environment (from the assignment code)
from Catch import CatchEnv

### Memory buffer, to store the trajectories

In [2]:
from collections import namedtuple, deque

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class MemoryBuffer(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity) # iterable deque data structure

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### Make a Deep Q-Network

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten



n_possible_actions = CatchEnv().get_num_actions
# the network should have 3 outputs, one for each of the possible moves

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



"""
This is broken as fuck, do not trust it, I copied it from somewhere :/
probably best to start over on this DQN model
"""

class DQN(nn.Module):

    def __init__(self, number_of_actions):
        super(DQN, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # conv layers
        self.conv1 = torch.nn.Conv2d(4, 32, 5)
        self.conv2 = torch.nn.Conv2d(32, 16, 3)
        # fully connected layers
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)   ## THE INPUT SHAPE TO THIS IS OFF (but I can't be bothered to calculate what it should be)
        self.fc2 = torch.nn.Linear(16 * 10 * 10, number_of_actions)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
# a function to train the model(s)?



## Loops

In [6]:
memoryBuffer = MemoryBuffer(100)

if torch.cuda.is_available():
    num_episodes = 500
else:
    num_episodes = 10

env = CatchEnv()

for ep in range(num_episodes):
    state = env.reset()
    state = np.squeeze(state)
    #state, reward, terminal = env.step(random.randint(0, 2))
    terminal = False
    while not terminal:

        # agent interacts with the environment
        action = random.randint (0, 2) # CHANGE, using the DQN model and 'state' to actually predict an action
        next_state, reward, terminal = env.step(action)
        next_state = np.squeeze(next_state)
        

        if terminal:
            next_state = None

        # add trajectory to memory buffer
        memoryBuffer.push(state, action, next_state, reward)

        # move onto the next state
        state = next_state


        # OPTIMISE THE NETWORK HERE
        """
        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        
        """


        #print ("  Reward obtained by the agent : {}". format(reward))
        
        #print(reward)
        #plt.imshow(env.image)
        #plt.show()

    print ("End of the episode")
    print ("  Reward obtained by the agent : {}". format(reward))

End of the episode
  Reward obtained by the agent : 1
End of the episode
  Reward obtained by the agent : 0
End of the episode
  Reward obtained by the agent : 0
End of the episode
  Reward obtained by the agent : 0
End of the episode
  Reward obtained by the agent : 0
End of the episode
  Reward obtained by the agent : 0
End of the episode
  Reward obtained by the agent : 0
End of the episode
  Reward obtained by the agent : 1
End of the episode
  Reward obtained by the agent : 0
End of the episode
  Reward obtained by the agent : 1
